In [5]:
import time
from adafruit_servokit import ServoKit

kit = ServoKit(channels=16)


mapp = {0:0,1:130,2:125}
# Reset all standard servos
# for ch in [7,13,14,15]:
#     try:
#         kit.servo[ch].angle = mapp[ch]  # release (no signal)
#     except Exception:
#         pass

# Reset all continuous servos
for ch in range(16):
    try:
        kit.continuous_servo[ch].throttle = 0  # stop
        time.sleep(0)

        # or use None to release completely
    except Exception:
        pass

print("✅ All channels reset to neutral/off")

✅ All channels reset to neutral/off


In [1]:
import time
import board
import busio
from adafruit_pca9685 import PCA9685
from adafruit_motor import servo
from adafruit_servokit import ServoKit

# -----------------------------
# PCA9685 setup
# -----------------------------
i2c = busio.I2C(board.SCL, board.SDA)
pca = PCA9685(i2c)
pca.frequency = 50  # standard servo frequency
kit = ServoKit(channels=16)  # For standard small servos

# -----------------------------
# Servo configuration
# -----------------------------
# Define which channels have large/high-torque vs small/RC servos
large_servos = [5,6]          # high-torque / metal gear
small_servos = [1, 3, 4] # standard hobby/RC servos

# Create servo objects for large servos with wide pulse range
large_servo_objects = {}
for ch in large_servos:
    s = servo.Servo(pca.channels[ch], min_pulse=500, max_pulse=2500)
    large_servo_objects[ch] = s
    s.angle = 0  # initialize to center
    time.sleep(0.05)

time.sleep(1)  # give all large servos time to reach position

# -----------------------------
# Helper functions
# -----------------------------
def move_to(servo_obj, current, target, step=2, delay=0.02):
    """
    Gradually move a large servo to the target angle
    """
    target = max(0, min(180, int(round(target))))
    if target == current:
        return current

    step = max(1, int(abs(step)))

    if target > current:
        angle = current
        while angle < target:
            angle = min(angle + step, target)
            servo_obj.angle = angle
            time.sleep(delay)
    else:
        angle = current
        while angle > target:
            angle = max(angle - step, target)
            servo_obj.angle = angle
            time.sleep(delay)

    return target  # updated current angle

def sweep_large_servo(ch):
    """
    Sweep a large servo on channel `ch` using gradual steps
    """
    s = large_servo_objects[ch]
    current_angle = 90
    current_angle = move_to(s, current_angle, 0, step=3, delay=0.04)
    current_angle = move_to(s, current_angle, 90, step=2, delay=0.04)
    current_angle = move_to(s, current_angle, 180, step=3, delay=0.04)
    current_angle = move_to(s, current_angle, 90, step=2, delay=0.04)
    current_angle = move_to(s, current_angle, 0, step=3, delay=0.04)

def sweep_small_servo(ch):
    """
    Sweep a small hobby/RC servo using direct jumps
    """
    print(f"Sweeping small servo on channel {ch}")
    
    for angle in [90, 180, 90, 0, 90]:
        kit.servo[ch].angle = angle
        time.sleep(1)  # give servo time to reach position

# -----------------------------
# Main program
# -----------------------------
try:
    print("=== Sweeping LARGE servos ===")
    for ch in large_servos:
        print(f"Channel {ch}")
        sweep_large_servo(ch)

    # print("\n=== Sweeping SMALL servos ===")
    # for ch in small_servos:
    #     sweep_small_servo(ch)

finally:
    print("\nCleaning up PCA9685")
    pca.deinit()


=== Sweeping LARGE servos ===
Channel 5
Channel 6

Cleaning up PCA9685


In [3]:
# 1 - j4 - mouth/hand/gripper
# 3 - j3 - wrist
# 4 - j2 - elbow
# 5 - j1 - shoulder
# 6 - j0 - waist
from adafruit_servokit import ServoKit

kit = ServoKit(channels=16)

# Dictionary to store last commanded positions
positions = {}

for ch in [1,3,4,5,6]:
    try:
        positions[ch] = kit.servo[ch].angle  # This is last set angle
    except Exception:
        positions[ch] = None

print(positions)

{1: 180.0, 3: -0.36600244001626675, 4: -0.36600244001626675, 5: -30.231801545343636, 6: -30.231801545343636}


In [11]:
import time
import board
import busio
from adafruit_pca9685 import PCA9685
from adafruit_motor import servo
from adafruit_servokit import ServoKit

# Startup angles from your INIT printout
startup_angles = {
    6: 90,
    5: 180,
    4: 0,
    3: 0,
    1: 180.0
}

# -----------------------------
# PCA9685 setup
# -----------------------------
i2c = busio.I2C(board.SCL, board.SDA)
pca = PCA9685(i2c)
pca.frequency = 50
kit = ServoKit(channels=16)

# -----------------------------
# Servo groups
# -----------------------------
large_servos = [5,6]
small_servos = [1, 3, 4]

# -----------------------------
# Create large servo objects
# -----------------------------
large_servo_objects = {}

for ch in large_servos:
    s = servo.Servo(pca.channels[ch], min_pulse=500, max_pulse=2500)
    large_servo_objects[ch] = s

    # set to real startup angle
    sa = max(0, min(180, int(startup_angles[ch])))
    s.angle = sa
    time.sleep(0.05)

time.sleep(0.3)


# -----------------------------
# Smooth movement helper
# -----------------------------
def move_to(servo_obj, current, target, step=2, delay=0.02):
    target = max(0, min(180, int(target)))
    current = max(0, min(180, int(current)))
    direction = 1 if target > current else -1

    for a in range(current, target + direction, direction * step):
        servo_obj.angle = a
        time.sleep(delay)

    return target


# -----------------------------
# Determine RELATIVE sweep path
# -----------------------------
def generate_sweep_targets(start):
    if start < 40:
        # near zero → go high first
        return [start, 180, 0, start]

    if start > 140:
        # near 180 → go low first
        return [start, 0, 180, start]

    # middle positions → symmetrical sweep
    return [start, 0, 180, start]


# -----------------------------
# Sweep functions
# -----------------------------
def sweep_large_servo(ch):
    start = startup_angles[ch]
    start = max(0, min(180, int(start)))
    s = large_servo_objects[ch]

    targets = generate_sweep_targets(start)

    print(f"[LARGE] ch{ch} sweep path: {targets}")

    current = targets[0]
    for target in targets[1:]:
        current = move_to(s, current, target, step=2, delay=0.05)


def sweep_small_servo(ch):
    start = startup_angles[ch]
    start = max(0, min(180, int(start)))

    targets = generate_sweep_targets(start)

    print(f"[SMALL] ch{ch} sweep path: {targets}")

    current = start
    for target in targets[1:]:
        kit.servo[ch].angle = target
        time.sleep(0.5)
        current = target


# -----------------------------
# MAIN PROGRAM
# -----------------------------
try:
    print("\n=== Sweeping LARGE servos with RELATIVE logic ===")
    for ch in large_servos:
        sweep_large_servo(ch)

    print("\n=== Sweeping SMALL servos with RELATIVE logic ===")
    for ch in small_servos:
        sweep_small_servo(ch)

finally:
    print("\nCleaning up PCA9685")
    # pca.deinit()



=== Sweeping LARGE servos with RELATIVE logic ===
[LARGE] ch5 sweep path: [180, 0, 180, 180]
[LARGE] ch6 sweep path: [90, 0, 180, 90]

=== Sweeping SMALL servos with RELATIVE logic ===
[SMALL] ch1 sweep path: [180, 0, 180, 180]
[SMALL] ch3 sweep path: [0, 180, 0, 0]
[SMALL] ch4 sweep path: [0, 180, 0, 0]

Cleaning up PCA9685


In [28]:
import time
import board
import busio
from adafruit_pca9685 import PCA9685
from adafruit_servokit import ServoKit

# --- Configuration ---

# Startup angles (The PCA9685 library uses angles 0-180 for standard servos)
# Note: Angles outside 0-180 are clamped by the function
STARTUP_ANGLES = {
    6: 90.0,
    5: 180.0,
    4: 0.0,
    3: 0.0,
    1: 180.0
}

# Servo channel groupings
LARGE_SERVOS = [5, 6]
SMALL_SERVOS = [1, 3, 4]

# Min/Max pulse definition (in microseconds)
# This is crucial for matching your specific servos and setting the 0°/180° limits.
# 500us to 2500us is a common range.
MIN_PULSE = 500
MAX_PULSE = 2500

# Movement speed parameters
SMOOTH_STEP = 1  # Angle increment per step (1 degree for very smooth)
SMOOTH_DELAY = 0.03  # Time delay between steps (0.01s for fast-smooth movement)

# --- PCA9685 Setup ---

# Initialize I2C connection
try:
    i2c = busio.I2C(board.SCL, board.SDA)
    pca = PCA9685(i2c)
    
    # Set the PWM frequency. 50Hz is standard. Higher may reduce audible hum, 
    # but 50Hz is often required for consistency. We'll stick to 50Hz.
    pca.frequency = 50 
    
    # Use ServoKit for simplified control over all channels
    kit = ServoKit(channels=16, i2c=i2c)
    
except ValueError as e:
    print(f"Error initializing I2C or PCA9685: {e}")
    print("Ensure wiring is correct and the I2C address is default (0x40).")
    exit()

# --- Servo Initialization and Pulse Range Setting ---

# Set the pulse width range for all active servos
all_channels = LARGE_SERVOS + SMALL_SERVOS

for ch in all_channels:
    # Set the physical limits of the servo pulse
    kit.servo[ch].set_pulse_width_range(min_pulse=MIN_PULSE, max_pulse=MAX_PULSE)
    
    # Set to initial startup angle
    # The max/min limits handle negative angles or values > 180 from your startup_angles dict
    start_angle = max(0.0, min(180.0, STARTUP_ANGLES.get(ch, 90.0)))
    kit.servo[ch].angle = start_angle
    time.sleep(0.05) # Small delay to let the servo initialize

print("Initialization complete. Servos at startup angles.")
time.sleep(0.5)

# ----------------------------------------------------
# Helper Functions
# ----------------------------------------------------

def move_to_smoothly(servo_kit_obj, current_angle, target_angle, step=SMOOTH_STEP, delay=SMOOTH_DELAY):
    """Moves a servo smoothly from current_angle to target_angle."""
    
    # Clamp target to the valid range (0-180)
    target_angle = max(0, min(180, target_angle))
    
    # Determine direction
    direction = 1 if target_angle > current_angle else -1

    # Use a small epsilon to handle float comparison and ensure loop termination
    if abs(target_angle - current_angle) < step:
        servo_kit_obj.angle = target_angle
        return target_angle

    # Loop through the path
    # We use round() for the range function but set the angle using floats 
    # for better resolution control if the library supports it.
    for angle in range(round(current_angle), round(target_angle) + direction * step, direction * step):
        servo_kit_obj.angle = float(angle)
        time.sleep(delay)

    # Ensure the final target is hit precisely
    servo_kit_obj.angle = target_angle
    return target_angle

def remove_pwm_signal(servo_kit_obj):
    """Stops the PWM pulse to the servo, eliminating holding hum/buzz."""
    # Setting the angle to None removes the signal entirely
    servo_kit_obj.angle = None
    print(f"-> Signal removed.")


def generate_sweep_targets(start_angle):
    """Determines a symmetrical or safe sweep path based on the start angle."""
    
    start = round(start_angle)

    if start < 40:
        # Near zero → go high first (0 → 180 → 0 → start)
        return [start, 180, 0, start]

    if start > 140:
        # Near 180 → go low first (180 → 0 → 180 → start)
        return [start, 0, 180, start]

    # Middle positions → symmetrical sweep (start → 0 → 180 → start)
    return [start, 0, 180, start]


def perform_sweep(ch):
    """Executes a full sweep on a single servo channel."""
    
    # Get the initial position and the servo object
    initial_angle = STARTUP_ANGLES.get(ch, 90.0)
    s_obj = kit.servo[ch]
    
    targets = generate_sweep_targets(initial_angle)

    print(f"[Channel {ch}] Sweep Path: {targets}")

    current = initial_angle
    for target in targets[1:]:
        print(f"Moving ch{ch} from {round(current)} to {target}...")
        current = move_to_smoothly(s_obj, current, target)
        time.sleep(0.5) # Pause at each target point

    # Final cleanup: stop the PWM signal to eliminate humming/buzzing
    # remove_pwm_signal(s_obj)


# ----------------------------------------------------
# MAIN EXECUTION
# ----------------------------------------------------

try:
    print("\n=== Starting Full Servo Sweep (Smooth Movement) ===")
    
    # Sweep large servos first
    for ch in LARGE_SERVOS:
        perform_sweep(ch)
        time.sleep(1.0) 

    # Sweep small servos
    for ch in SMALL_SERVOS:
        perform_sweep(ch)
        time.sleep(1.0) 

    print("\n=== All sweeps complete. PWM signal removed from all active channels. ===")
    
except Exception as e:
    print(f"\nAn error occurred: {e}")
    
finally:
    # Final cleanup is not strictly necessary but good practice 
    # to ensure the board is ready for the next run.
    print("\nCleaning up PCA9685...")
    # The PCA9685 object doesn't have a deinit() in the latest library, 
    # but we can stop all PWM outputs if needed.
    # The individual pulse removal above is usually sufficient.
    pass

Initialization complete. Servos at startup angles.



=== Starting Full Servo Sweep (Smooth Movement) ===
[Channel 5] Sweep Path: [180, 0, 180, 180]
Moving ch5 from 180 to 0...
Moving ch5 from 0 to 180...
Moving ch5 from 180 to 180...
[Channel 6] Sweep Path: [90, 0, 180, 90]
Moving ch6 from 90 to 0...
Moving ch6 from 0 to 180...
Moving ch6 from 180 to 90...
[Channel 1] Sweep Path: [180, 0, 180, 180]
Moving ch1 from 180 to 0...
Moving ch1 from 0 to 180...
Moving ch1 from 180 to 180...
[Channel 3] Sweep Path: [0, 180, 0, 0]
Moving ch3 from 0 to 180...
Moving ch3 from 180 to 0...
Moving ch3 from 0 to 0...
[Channel 4] Sweep Path: [0, 180, 0, 0]
Moving ch4 from 0 to 180...
Moving ch4 from 180 to 0...
Moving ch4 from 0 to 0...

=== All sweeps complete. PWM signal removed from all active channels. ===

Cleaning up PCA9685...


In [30]:
!sudo apt-get install python-smbus
!sudo apt-get install i2c-tools

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package python-smbus is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'python-smbus' has no installation candidate
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
i2c-tools is already the newest version (4.3-2+b3).
The following packages were automatically installed and are no longer required:
  avahi-utils gir1.2-handy-1 gir1.2-packagekitglib-1.0 gir1.2-polkit-1.0
  libbasicusageenvironment1 libc++1-16 libc++abi1-16 libcamera0.3 libcamera0.4
  libgroupsock8 liblivemedia77 libunwind-16 libwlroots12
  linux-headers-6.6.51+rpt-common-rpi linux-headers-6.6.51+rpt-rpi-2712
  linux-headers-6.6.51+rpt-rpi-v8 linux-image-6.6.51+rpt-rpi-2712
  linux-image-6.6.51+rpt-rpi-v8 linux-kbuild-6.6.51+rpt lxplug-network
  python3-v4l2
Us

In [2]:
!sudo i2cdetect -y 1


     0  1  2  3  4  5  6  7  8  9  a  b  c  d  e  f
00:                         -- -- -- -- -- -- -- -- 
10: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
20: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
30: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
40: 40 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
50: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
60: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
70: -- -- -- -- -- -- -- --                         


In [ ]:
from adafruit_servokit import ServoKit
kit = ServoKit(channels=16)

kit.servo[5].__dict__
print(kit.servo[5].angle=180)
kit.servo[5].angle = 209
kit.servo[5].angle = 210
kit.servo[5].angle = 211
kit.servo[5].angle = 212
kit.servo[5].angle = 213
kit.servo[5].angle = 214
kit.servo[5].angle = 213
kit.servo[5].angle = 212
kit.servo[5].angle = 211
kit.servo[5].angle = 210
kit.servo[5].angle = 209


209.86579910532737


ValueError: Angle out of range